**Aim-** Use TensorFlow to build our first Sequential model.


In [ ]:
import numpy as np
import pandas as pd


## Generate Dataset



In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

# Generate the data
X, y = datasets.make_regression(n_samples=100, n_features=10, noise=5, random_state=4)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

df= pd.DataFrame(X)
df['Output'] = y
df_train = pd.DataFrame(X_train)
df_train['Output'] = y_train
df_train_features = df_train.copy()
df_train_labels = df_train_features.pop('Output')

df_test = pd.DataFrame(X_test)
df_test['Output'] = y_test
df_test_features = df_test.copy()
df_test_labels = df_test_features.pop('Output')




## Visualize Dataset
This is the same code from Assignment 1

In [ ]:
import matplotlib.pyplot as plt

for i in range(X.shape[1]):
  plt.xlabel(i)
  plt.ylabel('Output')
  plt.scatter(df[i], df['Output'])
  plt.show()

# Your code here


[Note: Use this Link for help](https://www.tensorflow.org/tutorials/keras/regression)

## Model Definition

Using TensorFlow, build a model with the following definition:
> Input of shape 10 \\
> Dense of shape 50 \\
> Dense of shape 10 \\
Dense of shape 5 \\
> Dense of shape 1 \\

Use Mean Square Error Loss and Stochaistic Gradient Descent (SGD) Optimizer

Use Gradient Decay with appropriate parameters

In [ ]:
import tensorflow as tf
from tensorflow import keras

# normalization layer
norm = keras.layers.Normalization(axis=-1, mean=0, variance=1)
#optimizer
opt = keras.optimizers.experimental.SGD(learning_rate=0.01)

model = keras.Sequential([norm, keras.layers.Dense(50, activation='relu'), keras.layers.Dense(10, activation='relu'), keras.layers.Dense(5, activation='relu'), keras.layers.Dense(1)])
model.build(input_shape=(None,10))
model.compile(loss=keras.losses.MeanSquaredError(), optimizer=opt)
model.summary()

# Your code here

In [ ]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.xlabel('Epoch')
  plt.ylabel('Error [Output]')
  plt.legend()
  plt.grid(True)


In [ ]:
%%time
history_dnn = model.fit(
    df_train_features,
    df_train_labels,
    validation_split=0.2,
    verbose=0, epochs=500)

In [ ]:
history_dnn.history['val_loss']

## Plot Loss

Using matplotlib visualise how the loss (both validation and training) is changing, use this information to retrain the model with appropriate parameters.<br>We ideally want the loss to be constant over the last few iterations.

In [ ]:
# Your code here
plot_loss(history_dnn)

test_results={}
test_results['dnn_model'] = model.evaluate(df_test_features, df_test_labels, verbose=0)
pd.DataFrame(test_results, index=['Mean Square error']).T


## Evaluation Metrics
Use the R2 Score function implemented in the first assignment to evaluate the performance of the model.

In [ ]:
# Insert the function for R2 Score
def r2_score(y_true, y_pred):
    ssr= np.sum((np.subtract(y_true, y_pred))**2)
    mean = np.mean(y_true)
    sst = np.sum((np.subtract(mean, y_pred))**2)
    denom = ssr/sst
    r_score = 1-denom
    return r_score

In [ ]:
# predicting the results
pred = model.predict(df_test_features).flatten()
a = plt.axes(aspect='equal')
plt.scatter(df_test_labels, pred)
plt.xlabel('True Values')
plt.ylabel('Predictions')

In [ ]:
# checking accuracy of model
accuracy = r2_score(y_test, pred)
print(accuracy)

res = pd.DataFrame({'Expected': df_test_labels, 'Predicted': pred})
res
